# Import packages & dataset

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir())

# Any results you write to the current directory are saved as output.

['.ipynb_checkpoints', '05-902.pdf', '05-906 (1).pdf', '05-906.pdf', '1099-MISC_IronHorseInvestmentCoLLCDBACannon.zip', '1099-MISC_PhoenixRecyclingInc (1).zip', '1099-MISC_PhoenixRecyclingInc.zip', '1099MISC_IronHorseInvestmentCoLLCDBACannon.zip', '1099MISC_NEWCENTURYENTERPRISESINC.zip', '1099MISC_PhoenixRecyclingInc (1).zip', '1099MISC_PhoenixRecyclingInc.zip', '130-U.pdf', '2016 New Century p1 TR 001.jpg', '2017', '6560208132.pdf', '774742064358911.PDF', '780749350002-11025626.zip', '799482290002-11025626.zip', '857182110002-11025626.zip', 'Advertisement.pdf', 'affordability-ffm-exemption.pdf', 'Anaconda3-2019.03-Windows-x86_64.exe', 'April 2017 Bank Statement.pdf', 'Atlas 21.5 Quadra Color Manual.pdf', 'Atlas 22.75 Quadra Color Parts Catalog-2015.pdf', 'Attach27823_20171012_161207.jpg', 'Automobile_data.csv', 'business-bill-of-sale-purchase-agreement.docx', 'CCF02212019.pdf', 'CCI02212019.jpg', 'CDR_voice_7739875095 (1).csv', 'CDR_voice_7739875095.csv', 'Check Positive Pay Agreement

In [2]:
# Load the dataset
data = pd.read_csv('Automobile_data.csv')
# List the available columns
list(data)

['symboling',
 'normalized-losses',
 'make',
 'fuel-type',
 'aspiration',
 'num-of-doors',
 'body-style',
 'drive-wheels',
 'engine-location',
 'wheel-base',
 'length',
 'width',
 'height',
 'curb-weight',
 'engine-type',
 'num-of-cylinders',
 'engine-size',
 'fuel-system',
 'bore',
 'stroke',
 'compression-ratio',
 'horsepower',
 'peak-rpm',
 'city-mpg',
 'highway-mpg',
 'price']

In [18]:
data.columns

Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type',
       'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price'],
      dtype='object')

In [20]:
data.dtypes

symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horsepower           float64
peak-rpm               int64
city-mpg               int64
highway-mpg            int64
price                float64
dtype: object

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 204
Data columns (total 26 columns):
symboling            199 non-null int64
normalized-losses    199 non-null object
make                 199 non-null object
fuel-type            199 non-null object
aspiration           199 non-null object
num-of-doors         199 non-null object
body-style           199 non-null object
drive-wheels         199 non-null object
engine-location      199 non-null object
wheel-base           199 non-null float64
length               199 non-null float64
width                199 non-null float64
height               199 non-null float64
curb-weight          199 non-null int64
engine-type          199 non-null object
num-of-cylinders     199 non-null object
engine-size          199 non-null int64
fuel-system          199 non-null object
bore                 199 non-null object
stroke               199 non-null object
compression-ratio    199 non-null float64
horsepower           199 non-nul

In [3]:
# Preprocess the dataset by coercing the important columns to numeric values
data['horsepower'] = pd.to_numeric(data['horsepower'], 
                                   errors='coerce') # if value is not numeric, will fill in as NaN
data['price'] = pd.to_numeric(data['price'], 
                              errors='coerce')

# And removing any rows which contain missing data
data.dropna(subset=['price', 'horsepower'], 
            inplace=True)

# Statistically visualized the dataset

In [4]:
from scipy.stats.stats import pearsonr
pearsonr(data['horsepower'], data['price'])

# Pearson correlation to check the linear relationship between 2 variables, a number between -1 and 1
# more toward -1 negatively related, more toward 1 positively related, 0 no relation
# 2nd number is p-value, probability that 2 variables are NOT linearly related
# the smaller the lower the chance -> strongly indicated that 2 variables are related

(0.8105330821322063, 1.1891278276946011e-47)

In [5]:
# Graph the datapoints using bokeh to visualize linear relationship

from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource, figure, show

# enable notebook output
output_notebook()

source = ColumnDataSource(data=dict(
    x=data['horsepower'],
    y=data['price'],
    make=data['make'],
))

tooltips = [
    ('make', '@make'),
    ('horsepower', '$x'),
    ('price', '$y{$0}')
]

p = figure(plot_width=600, plot_height=400, tooltips=tooltips)
p.xaxis.axis_label = 'Horsepower'
p.yaxis.axis_label = 'Price'

# add a square renderer with a size, color, and alpha
p.circle('x', 'y', source=source, size=8, color='blue', alpha=0.5)

# show the results
show(p)

Loading BokehJS ...

# Simple Linear Regression

## Perform linear regression on 1 dependent and 1 independent variables

In [40]:
# split our data into train (75%) and test (25%) sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.25)

from sklearn import linear_model
model = linear_model.LinearRegression()

# the linear regression model expects a 2d array, so we add an extra dimension with reshape
# input: [1, 2, 3], output: [ [1], [2], [3] ]
# this allows us to regress on multiple independent variables later
training_x = np.array(train['horsepower']).reshape(-1, 1)
training_y = np.array(train['price'])

# perform linear regression
model.fit(training_x, training_y)

# output is a nested array in the form of [ [1] ]
# squeeze removes all zero dimensions -> [1]
# asscalar turns a single number array into a number -> 1
slope = np.asscalar(np.squeeze(model.coef_))
intercept = model.intercept_
print('slope:', slope, 'intercept:', intercept)

slope: 171.94201139055204 intercept: -4847.45544876098


In [41]:
model.coef_

array([171.94201139])

In [42]:
np.squeeze(model.coef_)

array(171.94201139)

In [43]:
model.intercept_

-4847.45544876098

In [44]:
# Now let's add the line to our graph
from bokeh.models import Slope
best_line = Slope(gradient=slope, y_intercept=intercept, line_color='red', line_width=3)
p.add_layout(best_line)
show(p)

In [45]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# define a function to generate a prediction and then compute the desired metrics
def predict_metrics(lr, x, y):
    pred = lr.predict(x)
    mae = mean_absolute_error(y, pred)
    mse = mean_squared_error(y, pred)
    r2 = r2_score(y, pred)
    return mae, mse, r2

training_mae, training_mse, training_r2 = predict_metrics(model, training_x, training_y)

# calculate with test data so we can compare
test_x = np.array(test['horsepower']).reshape(-1, 1)
test_y = np.array(test['price'])
test_mae, test_mse, test_r2 = predict_metrics(model, test_x, test_y)

print('training mean error:', training_mae, 'training mse:', training_mse, 'training r2:', training_r2)
print('test mean error:', test_mae, 'test mse:', test_mse, 'test r2:', test_r2)

training mean error: 3121.7263353998783 training mse: 20161208.698624592 training r2: 0.644533544086048
test mean error: 3818.1063303860415 test mse: 26785819.402696885 test r2: 0.6667589387914323


In [48]:
np.mean((model.predict(test_x) - test_y)**2) # mean square error

26785819.402696885

In [50]:
np.mean(np.abs(model.predict(test_x) - test_y)) # mean absolute error

3818.1063303860415

## Perform linear regression on 1 dependent variable and several independent variables

In [10]:
cols = ['horsepower', 'engine-size', 'peak-rpm', 'length', 'width', 'height']

# preprocess the data as before (coerce to number)
for col in cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# And removing any rows which contain missing data
data.dropna(subset=['price', 'horsepower'], inplace=True)

# Let's see how strongly each column is correlated to price
for col in cols:
    print(col, pearsonr(data[col], data['price']))

horsepower (0.8105330821322063, 1.1891278276946011e-47)
engine-size (0.8738869517981516, 1.2650674479074428e-63)
peak-rpm (-0.10164886620219901, 0.15311824317199588)
length (0.6939647745646871, 6.39831060305001e-30)
width (0.7538710519013427, 8.679834788813268e-38)
height (0.13499022754460993, 0.05730390719825449)


In [11]:
# split train and test data as before
model_cols = ['horsepower', 'engine-size', 'length', 'width']
multi_x = np.column_stack(tuple(data[col] for col in model_cols))
multi_train_x, multi_test_x, multi_train_y, multi_test_y = \
    train_test_split(multi_x, data['price'], test_size=0.25)

In [12]:
# fit the model as before
multi_model = linear_model.LinearRegression()

multi_model.fit(multi_train_x, multi_train_y)

multi_intercept = multi_model.intercept_
multi_coeffs = dict(zip(model_cols, multi_model.coef_))
print('intercept:', multi_intercept)
print('coefficients:', multi_coeffs)

intercept: -70324.0322629624
coefficients: {'horsepower': 52.445136577377546, 'engine-size': 88.2970830264895, 'length': 15.69069042110074, 'width': 976.185058271954}


In [13]:
# calculate error metrics
multi_train_mae, multi_train_mse, multi_train_r2 = predict_metrics(multi_model, multi_train_x, multi_train_y)
multi_test_mae, multi_test_mse, multi_test_r2 = predict_metrics(multi_model, multi_test_x, multi_test_y)
print('training mean error:', multi_train_mae, 'training mse:', multi_train_mse, 'training r2:', multi_train_r2)
print('test mean error:', multi_test_mae, 'test mse:', multi_test_mse, 'test r2:', multi_test_r2)

training mean error: 2376.368680804955 training mse: 10646771.454934087 training r2: 0.8365242209894131
test mean error: 2622.9424061106165 test mse: 14749658.867584916 test r2: 0.7442623980880259


# Ridge Regression

In [14]:
# Create train and test set
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.25)

# 1 dependent and 1 independent variable
X_train = np.array(train['horsepower']).reshape(-1, 1) # transform dataframe column to a 2D array
y_train = np.array(train['price']) # transform dataframe column to an array

# Import Ridge Regression
from sklearn import linear_model
model = linear_model.Ridge()

# Perform Ridge Regression
model.fit(X_train, y_train)

# Print result
slope = np.asscalar(np.squeeze(model.coef_))
intercept = model.intercept_
print('slope:', slope, 'intercept:', intercept)

# Calculate error metrics


slope: 180.0865818596351 intercept: -5330.4678016940725


In [15]:
# 1 dependent and several independent variables

# Choose several independent variables

# Check linear relationship between each independent variable and the dependent variable

# Remove independent variables that have poor linear relationship with the independent variable

# Create train and test set

# Run Ridge Regression

# Calculate error metrics

In [16]:
# Lasso Regression

In [17]:
# Calculate error metrics